In [8]:
import pandas as pd
import re
import itertools
import sqlite3
import json
import os

In [58]:
years = range(2000, 2024)
filenames = [f'/Users/trevor/Desktop/Research/climate-trends/climate_articles_{year}.json' for year in years]

In [59]:
common_fields = None  
for i, file in enumerate(filenames):
    file = pd.read_json(file)
    colnames = list(pd.DataFrame(list(itertools.chain(*file['message'].apply(lambda x: x['items'])))).columns)
    
    if common_fields is None:  
        common_fields = set(colnames)
    else: 
        common_fields = common_fields.intersection(set(colnames))

common_fields = list(common_fields)
print(common_fields)


['created', 'license', 'publisher', 'published-online', 'author', 'page', 'indexed', 'special_numbering', 'assertion', 'editor', 'relation', 'score', 'issued', 'volume', 'source', 'update-policy', 'updated-by', 'deposited', 'archive', 'is-referenced-by-count', 'alternative-id', 'prefix', 'DOI', 'language', 'abstract', 'resource', 'member', 'issue', 'link', 'ISSN', 'content-domain', 'published-print', 'type', 'short-container-title', 'journal-issue', 'references-count', 'subtitle', 'published-other', 'original-title', 'reference-count', 'published', 'title', 'container-title', 'reference', 'funder', 'issn-type', 'article-number', 'URL']


In [60]:
# Define import functions
def import_and_clean(path):
    file = pd.read_json(path)
    data = pd.DataFrame(list(itertools.chain(*file['message'].apply(lambda x: x['items']))))
    data['earliest_pub_year'] = data['published'].apply(lambda x: x['date-parts'][0][0])
    # Remove common junk words
    data['cleaned_abstract'] = data['abstract'].apply(lambda x: re.sub("<.*?>|\n|Abstract|abstract", "", str(x)))
    # Keep unique DOI's
    data.drop_duplicates(subset = 'DOI', keep ='first', inplace = True)
    # Subset to only English language articles
    data = data[data['language'] == 'en']
    # Subset to common columns
    data = data[common_fields +['cleaned_abstract', 'earliest_pub_year']]
    return data

In [61]:
def create_full_dataset():
    output_file = 'climate_articles_unique_english.csv'
    if os.path.exists(output_file):
        raise FileExistsError("The file {output_file} already exists. Ensure that additional information should not be written.")
    else:
        for i, filename in enumerate(filenames):
            file = import_and_clean(filename)
            if i == 0:
                file.to_csv(output_file, index = False, mode = 'w')
            else:
                file.to_csv(output_file, index = False, mode = 'a', header = False)

In [62]:
create_full_dataset()

In [63]:

full = pd.read_csv('climate_articles_unique_english.csv', on_bad_lines= 'warn')

/var/folders/1g/6pgh6bw14hgcbgppl7q0jgfr0000gn/T/ipykernel_1055/1484711313.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  full = pd.read_csv('climate_articles_unique_english.csv', on_bad_lines= 'warn')


In [64]:
full.shape

(328015, 50)

In [65]:
full_abs = full[full['abstract'].str.len() > 0]

In [66]:
full_abs.shape

(91197, 50)

In [67]:
full_abs.to_csv('unique_english_abstract_not_null.csv')